# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [62]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [176]:
print(set([element.tag for element in document.getiterator()]))

{'mondial', 'located_at', 'gdp_agri', 'elevation', 'city', 'mountain', 'infant_mortality', 'encompassed', 'ethnicgroup', 'depth', 'gmtOffset', 'border', 'gdp_serv', 'area', 'from', 'language', 'name', 'longitude', 'gdp_ind', 'abbrev', 'religion', 'inflation', 'located', 'river', 'gdp_total', 'mountains', 'through', 'country', 'government', 'to', 'island', 'length', 'localname', 'population_growth', 'lake', 'continent', 'unemployment', 'estuary', 'members', 'islands', 'sea', 'indep_date', 'organization', 'established', 'population', 'dependent', 'desert', 'airport', 'latitude', 'located_on', 'province', 'source'}


In [153]:
for country in document.getiterator('country'):
    if country.find('name').text == 'Greece':
        [print(child.tag) for child in country.getchildren()]

name
localname
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population_growth
infant_mortality
gdp_total
gdp_agri
gdp_ind
gdp_serv
inflation
unemployment
indep_date
government
encompassed
ethnicgroup
religion
religion
language
border
border
border
border
province
province
province
province
province
province
province
province
province
province
province
province
province
province


## Airport
I usually try to convert stuff to pandas dataframes for ease of use, but wanted to try using pure python here.

In [173]:
max_elevation = 0
for airport in document.getiterator('airport'):
    
    if airport.get('city'):
        country = airport.get('city').split('-')[1]
    if airport.find('elevation').text:
        elevation = int(airport.find('elevation').text)
    if elevation > max_elevation:
        max_country = country
        max_elevation = elevation
        max_airport = airport.find('name').text
        
print('The airport %s is the highest, at an elevation of %s' % (max_airport, max_elevation))
print('%s is in %s' % (max_airport, max_country))

The airport El Alto Intl is the highest, at an elevation of 4063
El Alto Intl is in BOL


## River

In [200]:
rivers = []
for river in document.getiterator('river'):
    name = river.find('name').text
    try:
        length = river.find('length').text
    except AttributeError as e:
        length = None

    countries = river.get('country').split(' ')
    
    rivers.append([name, length, countries])
    

In [202]:
rivers_df = pd.DataFrame(rivers).rename(columns={0:'name', 1:'length', 2:'countries'})
rivers_df.length = rivers_df.length.astype(float)
rivers_df.head()

,name,length,countries
0,Thjorsa,230.0,[IS]
1,Joekulsa a Fjoellum,206.0,[IS]
2,Glomma,604.0,[N]
3,Lagen,322.0,[N]
4,Goetaaelv,93.0,[S]


In [203]:
longest = rivers_df.sort_values(by='length', ascending=False).head(1)
longest

,name,length,countries
174,Amazonas,6448.0,"[CO, BR, PE]"


## Lake

In [210]:
lakes = []
for lake in document.getiterator('lake'):
    try:
        area = lake.find('area').text
    except AttributeError as e:
        area = None
        
    name = lake.find('name').text
    
    try:
        country = lake.find('located').get('country')
    except AttributeError as e:
        country = None
    
    lakes.append([name, area, country])
    
lakes_df = pd.DataFrame(lakes).rename(columns={0:'name', 1:'area', 2:'country'})
lakes_df.area = lakes_df.area.astype(float)
lakes_df.head()
        

,name,area,country
0,Inari,1040.0,SF
1,Oulujaervi,928.0,SF
2,Kallavesi,472.0,SF
3,Saimaa,4370.0,SF
4,Paeijaenne,1118.0,SF


In [211]:
biggest_lake = lakes_df.sort_values(by='area', ascending=False).head(1)
biggest_lake

,name,area,country
54,Caspian Sea,386400.0,R


## Infant Mortality by Country

In [60]:
infant_mortalities = {}
for country in document.getiterator('country'):
    country_name = country.find('name').text
    
    try:
        infant_mortality = country.find('infant_mortality').text
    except AttributeError as e:
        infant_mortality = None
    
    infant_mortalities[country_name] = infant_mortality

In [127]:
df = (pd.DataFrame.from_dict(infant_mortalities, orient='index')
        .rename(columns={0:'infant_mortality'})
        .sort_values(by='infant_mortality', ascending=False)
     )
top_10_im = df.head(10)
top_10_im

,infant_mortality
Central African Republic,92.86
Guinea-Bissau,90.92
Chad,90.3
Argentina,9.96
Thailand,9.86
Bahrain,9.68
Greenland,9.42
Botswana,9.38
Sint Maarten,9.05
Sri Lanka,9.02


## Top Population Cities and Top Ethnic Groups

In [137]:
def get_population(element):
    pops = {pop.get('year'): [pop.text, pop.get('measured')] for pop in element.findall('population')}
    pops_df = (pd.DataFrame.from_dict(pops, orient='index')
                .reset_index()
                .rename(columns={0:'population', 1:'meas_method', 'index':'year'})
              )
    return pops_df

def get_ethnic_groups(element):
    groups = {group.text: group.get('percentage') for group in element.findall('ethnicgroup')}
    groups_df = (pd.DataFrame.from_dict(groups, orient='index')
                             .reset_index()
                             .rename(columns={0:'percentage', 'index':'ethnic_group'})
                )
    return groups_df

In [133]:
dfs = []
for city in document.findall('country/city'):
    city_name = city.find('name').text
    pops_df = get_population(city)
    
    pops_df['city'] = city_name
    dfs.append(pops_df)
    
cities = pd.concat(dfs)
cities.population = cities.population.astype(int)

   
    

In [134]:
cities.head()

,city,meas_method,population,year
0,Tirana,None,192000,1987
1,Tirana,census,418495,2011
2,Tirana,estimate,244153,1990
0,Shkodër,None,62000,1987
1,Shkodër,census,77075,2011


In [125]:
most_recent_meas = (cities.sort_values(by=['city', 'year'])
                          .groupby('city')
                          .last()
                   )

most_recent_meas.head()


,meas_method,population,year
city,,,
Abomey-Calavi,census,307745,2002
Abu Dhabi,estimate,552000,2003
Aden,census,570551,2004
Akureyri,admin.,17490,2011
Al Ain,estimate,348000,2003


In [126]:
top_pop_cities = most_recent_meas.sort_values(by='population', ascending=False).head(10)
top_pop_cities

,meas_method,population,year
city,,,
Seoul,census,9708483,2010
Al Qahirah,census,8471859,2006
Bangkok,estimate,7506700,1999
Hong Kong,None,7055071,2009
Ho Chi Minh,census,5968384,2009
Singapore,census,5076700,2010
Al Iskandariyah,census,4123869,2006
New Taipei,estimate,3939305,2012
Busan,census,3403135,2010


In [143]:
pops_dfs, ethnic_group_dfs = [], []
for country in document.findall('country'):
    country_name = country.find('name').text
    pops = get_population(country)
    pops['country'] = country_name
    pops_dfs.append(pops)
    
    ethnic_groups = get_ethnic_groups(country)
    ethnic_groups['country'] = country_name
    ethnic_group_dfs.append(ethnic_groups)
    
pops = pd.concat(pops_dfs)
pops.population = pops.population.astype(int)

ethnic_groups = pd.concat(ethnic_group_dfs)
ethnic_groups.percentage = ethnic_groups.percentage.astype(float)/100
    



In [144]:
ethnic_groups.head()

,country,ethnic_group,percentage
0,Albania,Albanian,0.950
1,Albania,Greek,0.030
0,Greece,Greek,0.930
0,Macedonia,Serb,0.018
1,Macedonia,Turkish,0.039


In [146]:
most_recent_pop = (pops.sort_values(by=['country', 'year'])
                      .groupby('country')
                      .last()
                   )
most_recent_pop.head()

,year,population,meas_method
country,,,
Afghanistan,2013,26023100,estimate
Albania,2011,2800138,census
Algeria,2010,37062820,est.
American Samoa,2010,55519,census
Andorra,2011,78115,admin.


In [151]:
m = pd.merge(ethnic_groups, most_recent_pop.reset_index(), how='left', on='country')
m['population'] = m.population * m.percentage
m.head()

,country,ethnic_group,percentage,year,population,meas_method
0,Albania,Albanian,0.950,2011,2.660131e+06,census
1,Albania,Greek,0.030,2011,8.400414e+04,census
2,Greece,Greek,0.930,2011,1.005915e+07,census
3,Macedonia,Serb,0.018,2011,3.707629e+04,estimate
4,Macedonia,Turkish,0.039,2011,8.033197e+04,estimate


In [159]:
ethnic_group_pops = m.groupby('ethnic_group').population.sum()
top_ethnic_groups = ethnic_group_pops.sort_values(ascending=False).head(10)
top_ethnic_groups
                     

ethnic_group
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: population, dtype: float64